# Поиск "токсичных" комментариев.


# Описание проекта

Нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. В наличии есть набор данных с разметкой о токсичности комментариев.

In [1]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from catboost import CatBoostClassifier, Pool

##  Загрузка и обработка данных

In [2]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv',index_col=[0])
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Посмотрим на целевой признак.

In [3]:
data['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Явно тут дисбаланс классов.

Проверим явные дубликаты.

In [4]:
data.duplicated().sum()

0

Удалим лишние символы

In [5]:
data['text'] = data['text'].apply(lambda x: re.sub(r'[^a-zA-Z]',' ',x))

Избавимся от пробелов

In [6]:
data['text'] = data['text'].apply(lambda x: ' '.join(x.split()) )

In [7]:
data.head()

,text,toxic
0,Explanation Why the edits made under my userna...,0
1,D aww He matches this background colour I m se...,0
2,Hey man I m really not trying to edit war It s...,0
3,More I can t make any real suggestions on impr...,0
4,You sir are my hero Any chance you remember wh...,0


Проведем лемматизацию текста. Используем библиотеку nltk. Чтобы поменялись глаголы мспользуем pos_tag из nltk.

In [8]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
wnl = WordNetLemmatizer()

data['text_lemm'] = data['text'].apply(lambda x: ' '.join([wnl.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','n','v'] else wnl.lemmatize(i) for i,j in pos_tag(word_tokenize(x))]))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Sergey.Polushkin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sergey.Polushkin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
data.head()

,text,toxic,text_lemm
0,Explanation Why the edits made under my userna...,0,Explanation Why the edits make under my userna...
1,D aww He matches this background colour I m se...,0,D aww He match this background colour I m seem...
2,Hey man I m really not trying to edit war It s...,0,Hey man I m really not try to edit war It s ju...
3,More I can t make any real suggestions on impr...,0,More I can t make any real suggestion on impro...
4,You sir are my hero Any chance you remember wh...,0,You sir be my hero Any chance you remember wha...


Глаголы поменялись!

In [10]:
data['text_lemm'] = data['text_lemm'].str.lower()
#corpus = data['text_lemm'].values.astype('U')

Получим список стоп-слов для английского языка.

In [11]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sergey.Polushkin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Обучение моделей

Разделим данные на обучающую и тестовые выборки. Оказывается random_state везде в примерах ставят 42, потому что это ответ на вопрос "В чём смысл жизни и вообще?" из Автостопом по галактике.

In [12]:
X_train, X_test, y_train, y_test = train_test_split(data['text_lemm'], data['toxic'], test_size=0.25, random_state=42,stratify=data['toxic'])

Получим TF-IDF для корпуса текста

In [13]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
tf_idf_train = count_tf_idf.fit_transform(X_train)
tf_idf_test = count_tf_idf.transform(X_test)

### Логистическая регрессия

In [14]:
reg = LogisticRegressionCV(cv=3,scoring='f1', class_weight='balanced' ,solver='liblinear', random_state=42).fit(tf_idf_train, y_train)

In [15]:
reg.score(tf_idf_train, y_train)

0.8687608821822403

In [16]:
valid_dict = {}
valid_dict['LogisticRegression'] = [reg.score(tf_idf_train, y_train)]

### Случайный лес

In [17]:
parameters = {'n_estimators':[50,70,100],'max_depth':[5,10,15]}
forest = RandomForestClassifier(random_state=42,class_weight='balanced')
forest_clf = GridSearchCV(forest, parameters,cv=3,scoring='f1',verbose=10)
forest_clf.fit(tf_idf_train, y_train)
forest_clf.best_estimator_

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 1/3; 1/9] START max_depth=5, n_estimators=50................................
[CV 1/3; 1/9] END ..............max_depth=5, n_estimators=50; total time=   4.9s
[CV 2/3; 1/9] START max_depth=5, n_estimators=50................................
[CV 2/3; 1/9] END ..............max_depth=5, n_estimators=50; total time=   5.4s
[CV 3/3; 1/9] START max_depth=5, n_estimators=50................................
[CV 3/3; 1/9] END ..............max_depth=5, n_estimators=50; total time=   5.0s
[CV 1/3; 2/9] START max_depth=5, n_estimators=70................................
[CV 1/3; 2/9] END ..............max_depth=5, n_estimators=70; total time=   6.7s
[CV 2/3; 2/9] START max_depth=5, n_estimators=70................................
[CV 2/3; 2/9] END ..............max_depth=5, n_estimators=70; total time=   6.8s
[CV 3/3; 2/9] START max_depth=5, n_estimators=70................................
[CV 3/3; 2/9] END ..............max_depth=5, n_es

RandomForestClassifier(class_weight='balanced', max_depth=15, random_state=42)

In [18]:
forest_clf.best_score_

0.3667450502786933

In [19]:
valid_dict['RandomForest'] = [forest_clf.best_score_]

### Catboost

In [20]:
train_pool = Pool(tf_idf_train, y_train)
test_pool = Pool(tf_idf_test) 

In [21]:
model = CatBoostClassifier(logging_level='Silent',eval_metric='TotalF1',class_weights=[0.1,0.9],learning_rate=0.1)

In [22]:
grid = {'depth': [10],
        'iterations':[70, 100]

        }

In [23]:
grid_search_result = model.grid_search(grid,
            train_pool,
            cv=3,
            verbose=False,
            plot=True
            )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [24]:
max(grid_search_result['cv_results']['test-TotalF1-mean'])

0.8665582458368265

In [25]:
valid_dict['CatBoost'] = [max(grid_search_result['cv_results']['test-TotalF1-mean'])]

In [26]:
grid_search_result['params']

{'depth': 10, 'iterations': 100}

## Тестирование

In [27]:
pd.DataFrame.from_dict(valid_dict,orient='index',columns=['F1']).round(decimals=2).sort_values(by='F1',ascending=False)

,F1
LogisticRegression,0.87
CatBoost,0.87
RandomForest,0.37


Наилучший результат показала логистическая регрессия. Проверим её на тестовой выборке.

In [31]:
y_test_predict = reg.predict(tf_idf_test)

In [32]:
f1_score(y_test, y_test_predict)

0.7567567567567567

Логистическая регрессия прошла проверку.

Проверим модель на адекватность. Пусть наша модель предсказывает, что все отзывы негативные.

In [34]:
f1_score([1]*len(y_test),y_test)

0.18445827349609065

А если все отзывы положительные?

In [35]:
f1_score([0]*len(y_test),y_test)

0.0

Наша модель определенно работает лучше.

## Вывод
- Логистическая регрессия показала наилучший результат и прошла пороговое значение в 0.75 на тестовой выборке. И обучается она занчительно быстрее, чем остальные методы. Оставляем логистическую регрессию.
- Если брать все отзывы негативными, то занчение F1 получается значительно меньше, чем в логистической регрессии, значит наша модель умеет достаточно хорошо предсказывать тип отзыва. 